## Quantile Model Prediction - V2
Predict seasonal flow volumes using combination of snow pack, antecedent flow models and quantile regression model.

In [8]:
import pandas as pd
from sklearn.linear_model import QuantileRegressor, LinearRegression
import datetime
from typing import Literal, Tuple
import pickle

#### Load SNODAS, antecedent flow, and submission csv

In [2]:
snodas_data = pd.DataFrame()
for year in range(2005,2024):
    year_data = pd.read_csv(f'./data/snodas/snodas_swe_{year}.csv', index_col=0, parse_dates=True)
    snodas_data = pd.concat([snodas_data, year_data])

In [3]:
model_submission = pd.read_csv('./data/competition/submission_format.csv')

In [4]:
monthly_flow = pd.read_csv('./data/competition/test_monthly_naturalized_flow.csv')

In [5]:
metadata = pd.read_csv('./data/competition/metadata.csv').set_index('site_id')

In [6]:
# Change the column names of the SNODAS data to match the other datasets
snodas_data.columns = metadata.index.to_numpy()

#### Load pre-trained models

In [7]:
# Unpickle the models
with open('./model/snow_models.pkl', 'rb') as f:
    snow_models = pickle.load(f)

with open('./model/antecedent_flow_models.pkl', 'rb') as f:
    antecedent_flow_models = pickle.load(f)

with open('./model/total_flow_quantile_models.pkl', 'rb') as f:
    total_flow_models = pickle.load(f)

#### Assess antecedent flow data
I visually verifed that the test data is cleaner than the training data and has no nans or missing values.

In [9]:
# Algorithm to determine months of antecedent flow. The water year starts in October, so antecedent flow for a given month is every month of the water year before that month.
water_year_months = [10,11,12,1,2,3,4,5,6,7,8,9]
def antecedent_months(month: int) -> Tuple[int]:
    return tuple(water_year_months[:water_year_months.index(month)])

In [12]:
# Create a function for getting total antecedent flow for a given site and month. Return series of years.
# In order to keep the same interface, I will include a day parameter. So for example, total flow on 2/15 will be total flow for January.
def get_antecedent_flow(site_id: str, day: int, month: int, year: int) -> float:
    site_flow_data: pd.DataFrame = monthly_flow.loc[monthly_flow['site_id'] == site_id,:].copy()
    site_flow_data['antecedent'] = site_flow_data['month'].apply(lambda x: True if x in antecedent_months(month) else False)

    # Calculate total flow where is_runoff is True by site and year
    site_flow_totals = site_flow_data[site_flow_data['antecedent']].groupby(['site_id','forecast_year']).sum().reset_index()

    site_flow_totals.set_index('forecast_year', inplace=True)
    
    return site_flow_totals.loc[year,'volume']

# Test the function at the pecos_r_nr_pecos
print(get_antecedent_flow('pecos_r_nr_pecos', 15, 1,2019))

6.335


In [13]:
# Identify sites that have no antecedent flow data.
sites_no_antecedent = [site for site in snodas_data.columns if site not in monthly_flow['site_id'].unique()]

### Create a function to perform the prediction

In [24]:
def predict_quantile(site:str, prediction_date_str: str, quantile: Literal['0.1','0.5','0.9']):
    """
    Predicts the total seasonal volumn for a given site using the snow and antecedent flow models as input to the total flow model.

    Sites without antecedent flow data will use the snow model only.

    Any dates that are after the runoff start date use the value for the runoff start date.
    """
    # Convert the prediction date to a datetime object
    prediction_date = datetime.datetime.strptime(prediction_date_str, '%Y-%m-%d').date()

    # Check if the prediction date is on or after the runoff start date
    # If it is, use the runoff start data and don't use the snow or antecedent flow models
    is_after_season_start = False
    season_start_month = metadata.loc[site,'season_start_month']
    if prediction_date.month >= season_start_month:
        prediction_date = datetime.date(prediction_date.year, season_start_month, 1)
        is_after_season_start = True

    # First determine season start SWE
    if is_after_season_start:
        # Handle missing 2017-04-01 from SNODAS data
        if prediction_date == datetime.date(2017,4,1):
            prediction_date = datetime.date(2017,4,8)

        # Get the snodas data for the site and date and convert to KAF
        season_start_swe = snodas_data.loc[prediction_date.strftime('%Y-%m-%d'), site] / 1233.48
    else:
        # Get the model for the site and date
        site_snow_model: LinearRegression = snow_models[site][(prediction_date.month, prediction_date.day)]
        current_swe = snodas_data.loc[prediction_date.strftime('%Y-%m-%d'), site]

        # Predict season start SWE
        season_start_swe = site_snow_model.predict([[current_swe]])[0][0]
        season_start_swe = season_start_swe/1233.48 #KAF

    # Second determine antecedent flow for sites with antecedent flow data
    antecedent_flow = None
    if site not in sites_no_antecedent:
        if is_after_season_start:
            antecedent_flow = get_antecedent_flow(site, prediction_date.day, prediction_date.month, prediction_date.year)
        else:
            site_antecedent_flow_model: LinearRegression = antecedent_flow_models[site][(prediction_date.month, prediction_date.day)]
            current_antecedent_flow = get_antecedent_flow(site, prediction_date.day, prediction_date.month, prediction_date.year)
            antecedent_flow = site_antecedent_flow_model.predict([[current_antecedent_flow]])[0][0]

    # Finally predict total flow from swe and antecedent flow
    total_flow_model: QuantileRegressor = total_flow_models[site][quantile]
    if site in sites_no_antecedent:
        total_flow = total_flow_model.predict([[season_start_swe]])[0]
    else:
        total_flow = total_flow_model.predict([[season_start_swe, antecedent_flow]])[0]

    return total_flow

In [25]:
# Test the model
test = predict_quantile('pecos_r_nr_pecos', '2023-01-01', '0.5')
print(test)

20.16835931011733


c:\Users\clmbn\CRCEAnalytics\WaterSupplyComp\.drivendata\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but QuantileRegressor was fitted with feature names
  warnings.warn(


#### Predict quantiles for each site
Use the submission format to generate all the predictions

In [ ]:
model_submission['volume_10'] = model_submission.apply(lambda row: predict_quantile(row['site_id'], row['issue_date'], '0.1'), axis=1)

In [ ]:
model_submission['volume_50'] = model_submission.apply(lambda row: predict_quantile(row['site_id'], row['issue_date'], '0.5'), axis=1)

In [ ]:
model_submission['volume_90'] = model_submission.apply(lambda row: predict_quantile(row['site_id'], row['issue_date'], '0.9'), axis=1)

In [29]:
# Save the submission to csv
model_submission.to_csv('./data/competition/submissions/quantileV2_submission_122023.csv', index=False)